In [21]:

# pytorch mlp for regression
from numpy import vstack
import numpy as np
from numpy import sqrt
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Tanh
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.optim import Adam
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_
 

# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, :-1].astype('float32')
        self.y = df.values[:, -1].astype('float32')
        # print(len(self.X[0]))
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 1))
        
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])
 
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs,layer1, layer2, act_func):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, layer1)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = act_func()
        # second hidden layer
        self.hidden2 = Linear(layer1, layer2)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = act_func()
        # third hidden layer and output
        self.hidden3 = Linear(layer2, 1)
        xavier_uniform_(self.hidden3.weight)
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        return X
 
# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    print(CSVDataset(path).__getitem__(0))
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl
 
# train the model
def train_model(train_dl, model, learn):
    # define the optimization
    criterion = MSELoss()
    optimizer = Adam(model.parameters(), lr=learn)
    # enumerate epochs
    for epoch in range(1000):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
 
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals, erroDist = list(), list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    
    predictions, actuals = vstack(predictions), vstack(actuals)
  
    predit = np.array(predictions)
    actu = np.array(actuals)
    act = actu.tolist()
    dif = predit-actu.tolist()
    
    j=0
    while j < len(dif):
        erroDist.append([dif[j],act[j]])
        with open('erros_dist1.csv', 'a') as f:
                f.write(f'{dif[j][0]},{act[j][0]}\n')
        j+=1
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    mae = mean_absolute_error(actuals, predictions)
    menor = min(dif)
    maior = max(dif)
    # print(f'menor erro: {min(dif)}')
    # print(f'maior erro: {max(dif)}')
    
    return mse, mae, menor, maior, erroDist
 
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat
 
# prepare the data
path = 'DataSet1 - Beacon1.csv'
train_dl, test_dl = prepare_data(path)
# print(len(train_dl.dataset), len(test_dl.dataset))
# layers1 = 130
# layers2 = 30
# actFunc = ReLU
# learning = 0.001
# model = MLP(10,layers1, layers2, actFunc)
# evaluate_model(train_dl,model)

[array([-43., -48., -48., -48., -49., -45., -48., -49., -51., -49.],
      dtype=float32), array([0.5], dtype=float32)]


In [20]:

 
layers1 = 130
layers2 = 30
actFunc = ReLU
learning = 0.001

# define the network

model = MLP(10,layers1, layers2, actFunc)
# train the model
train_model(train_dl, model,learning)
# evaluate the model


In [22]:
print(len(train_dl))

38


In [24]:
mse, mae, menorErro, maiorErro, erroDist = evaluate_model(train_dl, model)
print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))
print('MAE: %.3f' % (mae))
# print(erroDist)
print(f'erro{menorErro}')
print(f'erro{maiorErro}')

# make a single prediction (expect class=1)
row = [-56,-54,-60,-54,-58,-60,-55,-59,-59,-54]
yhat = predict(row, model)
print('Predicted: %.3f' % yhat)

MSE: 1.011, RMSE: 1.005
MAE: 0.762
erro[-3.93185246]
erro[2.65860271]
Predicted: 4.852


In [ ]:
import pandas as pd

df = pd.read_csv('results11.csv')

df=df.sort_values(by=['mse'])

df.to_csv('resultsOrd11.csv',index=False)